In [2]:
import numpy as np
import random as rd
import math
GENERATIONS = 100
POPULATION_SIZE = 1000
CHROMOSOME_SIZE = 4
MUTATION_RATE = 0.2

VOICES_LIMIT = 275
DATAS_LIMIT = 110

P_ELIT = 0.5

#Limites estabelecidos para cada gente
CromLimVoice_GSM = [0,125]
CromLimVoice_WCDMA = [0,150]
CromLimData_GSM= [0,30]
CromLimData_WCDMA= [0,80]


class Chromosome():
      def __init__(self):
          self.sequence = []
          self.quantization = []
          self.binary = ''
          self.fitness = None
      def setSequence(self, val):
        self.sequence = val
      def setFitness(self, fitness):
        self.fitness = fitness
      def setBinary(self, val):
        self.binary = val

def decimalToBinary(n):
    binary = "{0:b}".format(int(n))
    if (len(binary) < 9):
      binary = binary.zfill(9)
    return binary

def fitness(chromosome):
    userData = 1 - (chromosome[1] + chromosome[3])/110
    userVoice = 1 - (chromosome[0] + chromosome[2])/275
    gsmCost = abs(-30 + chromosome[1] + 6/25*chromosome[0])
    wcdmaCost = abs(-80 + chromosome[3] + 8/15*chromosome[2])
    return (pow(gsmCost, 2) + pow(wcdmaCost, 2)) * userData * userVoice


def sortPopulation(population):
    population.sort(key=lambda x: x.fitness)
    return population


def generatePopulation():
    population = [Chromosome() for i in range(POPULATION_SIZE)]
    i = 0
    while i < POPULATION_SIZE:
        population[i].setSequence([rd.uniform(CromLimVoice_GSM[0], CromLimVoice_GSM[1]),
                                   rd.uniform(CromLimData_GSM[0], CromLimData_GSM[1]),
                                   rd.uniform(CromLimVoice_WCDMA[0],CromLimVoice_WCDMA[1]),
                                   rd.uniform(CromLimData_WCDMA[0], CromLimData_WCDMA[1])])
        population[i].setFitness(fitness(population[i].sequence))
        i+=1

    return population


def isValid(chromosome):
  sumVoices = chromosome.sequence[0] + chromosome.sequence[2]
  sumDatas  = chromosome.sequence[1] + chromosome.sequence[3]

  if sumVoices > VOICES_LIMIT:
    return False
  if sumDatas > DATAS_LIMIT:
    return False

  return True


def quantVoiceGSM(gene):
  return gene / 125 * 2**9 - 1


def quantVoiceWCDMA(gene):
  return gene / 150 * 2**9 - 1


def quantDataGSM(gene):
  return gene / 30 * 2**9 - 1


def quantDataWCDMA(gene):
  return gene / 80 * 2**9 - 1


def desQuantVoiceGSM(gene):
  return gene * 125 / 2**9 - 1


def desQuantDataGSM(gene):
  return gene * 30 / 2**9 - 1


def desQuantVoiceWCDMA(gene):
  return gene * 150 / 2**9 - 1


def desQuantDataWCDMA(gene):
  return gene * 80 / 2**9 - 1


def getParents(population):
  fitness = [1-p.fitness for p in population]
  total = sum(1-p.fitness for p in population)
  probabilities = [fitness/total for fitness in fitness]

  one, two = rd.choices(population, probabilities, k=2)
  
  return [one, two]


def remove(population):
  values = P_ELIT * CHROMOSOME_SIZE
  while values > 0:
    population.pop(-1)
    values -= 1
  return population


def getQuantization(population):
  for idx, p in enumerate(population):
    binaryA = decimalToBinary(round(quantVoiceGSM(p.sequence[0]), 3))
    binaryB = decimalToBinary(round(quantDataGSM(p.sequence[1]), 3))
    binaryC = decimalToBinary(round(quantVoiceWCDMA(p.sequence[2]), 3))
    binaryD = decimalToBinary(round(quantDataWCDMA(p.sequence[3]), 3))
    p.binary = binaryA + binaryB + binaryC + binaryD


def desQuantization(sequence):
      quantA = sequence.binary[:9]
      quantB = sequence.binary[9:18]
      quantC = sequence.binary[18:27]
      quantD = sequence.binary[27:35]

      VoiceGSM = round(desQuantVoiceGSM(float(quantA)),3)
      DataGSM = round(desQuantDataGSM(float(quantB)),3)
      VoiceWCDMA = round(desQuantVoiceWCDMA(float(quantC)),3)
      DataWCDMA = round(desQuantDataWCDMA(float(quantD)),3)

      sequence.setSequence([VoiceGSM, DataGSM, VoiceWCDMA, DataWCDMA])
      sequence.setFitness(fitness(sequence.sequence))

      return sequence


def crossover(parents, population):
    one, two = parents
    value = rd.randint(0, len(one.binary))

    one.setBinary(one.binary[:value] + two.binary[value:])
    two.setBinary(two.binary[:value] + one.binary[value:])

    new_one = desQuantization(one)
    new_two = desQuantization(two)

    if isValid(new_one):
      population.append(new_one)
    if isValid(new_two):
      population.append(new_two)
    
    return population


def mutation(population):
  population = sortPopulation(population)
  removed = P_ELIT * CHROMOSOME_SIZE

  for idx in range(1, len(population)):
    total_crom = len(population) + removed
    held_crom = len(population)
    values_bits = rd.randint(1, 18)
    bits_mut = math.ceil((total_crom-held_crom) * values_bits * MUTATION_RATE)

    selected_indexes = rd.sample(range(len(population[idx].binary)-1), bits_mut)

    for current_index in selected_indexes:
      current_binary_item = str(1 - int(population[idx].binary[current_index]))

      new_binary = list(population[idx].binary)
      new_binary[current_index] = current_binary_item
      population[idx].binary = ''.join(new_binary)


if __name__ == '__main__':
    population = generatePopulation()

    while GENERATIONS >= 0:
      getQuantization(population)
      population = sortPopulation(population)

      population = remove(population)
      if len(population) <= 2: break
      parents = getParents(population)
      population = crossover(parents, population)

      mutation(population)
      GENERATIONS -= 1

    print(f"Melhor sequência encontrada = {[round(x, 2) for x in population[0].sequence]}")
    print(f'Fitness da sequência = {round(population[0].fitness, 2)}')

Melhor sequência encontrada = [16.27, 25.9, 112.97, 19.99]
Fitness da sequência = 0.03
